In [2]:
import pandas as pd
import numpy as np

import yfinance as yf


import requests
from datetime import datetime, timedelta, timezone


import warnings

warnings.filterwarnings('ignore')

MONTH_DIR = 'Mar 2025'


In [4]:
# Stables
# 31566704 USDC
# 312769 USDT 
# 227855942 EURS
# 760037151 xUSD


In [5]:
def download_csv(coin_id, currency):
    """
    This function downloads the data from Coingecko with the usd pair.
    Inputs:
        - coin_id: the name of the crypto we are getting the data
        - currency: by default it is the usd
    Output:
        - This function simply stores under raw_data directory a file
        from coingecko with all historic data available
    """
    # Construct the URL
    base_url = "https://www.coingecko.com"
    csv_path = f"/price_charts/export/{coin_id}/{currency}.csv"
    full_url = base_url + csv_path

    # Download the CSV
    response = requests.get(full_url)
    if response.status_code == 200:
        file_name = f"{MONTH_DIR}/{coin_id}_historical_data.csv"
        with open(file_name, "wb") as file:
            file.write(response.content)

    else:
        print(f"Failed to download CSV for {coin_id}.")

download_csv('algorand', 'usd')

In [3]:
def date_to_unix_timestamp(start_date_str, end_date_str):
  """
  Converts start and end dates in YYYY-MM-DD format to Unix timestamps.

  Args:
    start_date_str: Start date string in YYYY-MM-DD format.
    end_date_str: End date string in YYYY-MM-DD format.

  Returns:
    A tuple containing the start and end Unix timestamps.
  """

  start_date = datetime.strptime(start_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
  end_date = datetime.strptime(end_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc) + timedelta(days=1) - timedelta(seconds=1) 

  start_timestamp = int(start_date.timestamp())
  end_timestamp = int(end_date.timestamp())

  return start_timestamp, end_timestamp

In [4]:
def get_close_price(start_date, end_date, asset_id):
    """
    Fetches an asset historical data from the API for the given interval.
    
    Args:
        start_timestamp: start unix timestamp
        end_timestamp: end unix timestamp

    Returns:
        pd.DataFrame: Dataframe containing the fetched data in daily intervals.
    """
    start_unix, end_unix = date_to_unix_timestamp(start_date, end_date)

    price_feed = f'https://indexer.vestige.fi/assets/{asset_id}/candles?network_id=0&interval=86400&start={start_unix}&end={end_unix}&denominating_asset_id=0&volume_in_denominating_asset=false'

    response = requests.get(price_feed)
    data = response.json()
    df = pd.DataFrame(data)
    return df

In [5]:
start_date = '2025-03-01'
end_date = '2025-03-31'

stables_dfs = {}
stables_ids = {'usdc': 31566704, 'usdt': 312769,
              'eurs': 227855942,'xusd':760037151}

for stablecoin, stables_id in stables_ids.items():
    stables_dfs[stablecoin] = get_close_price(start_date, end_date, stables_id)




In [6]:
start_date = '2025-03-01'
end_date = '2025-03-31'

asas_dfs = {}
asas_ids = {
            'ora': 1284444444, 'niko': 1265975021, 'talgo': 2537013734, 'vote': 452399768, 
            'tiny': 2200000000, 'gobtc': 386192725, 'silver': 246519683, 'rio': 2751733, 
            'stbl': 465865291, 'alpha': 2726252423, 'monko': 2494786278, 'opul': 287867876, 
            'gold': 246516580, 'chip': 388592191, 'malgo': 1185173782, 'coop': 796425061, 
            'avoi': 2320775407, 'goeth': 386195940, 'daffir': 1268830233, 'calgo': 2400334372
        }

for asa, asa_id in asas_ids.items():
    asas_dfs[asa] = get_close_price(start_date, end_date, asa_id)

In [8]:
def create_combined_df(dataframes):
  """
  Creates a single DataFrame with 'date' as the index and 
  'close' columns named after the stablecoin keys.

  Args:
    dataframes: A dictionary where keys are stablecoin names 
                and values are the corresponding DataFrames.

  Returns:
    A pandas DataFrame with the combined data.
  """

  combined_df = pd.DataFrame()

  for asset, df in dataframes.items():
    df['date'] = pd.to_datetime(df['timestamp'], unit='s') 
    df.set_index('date', inplace=True) 
    df.rename(columns={'close': asset}, inplace=True) 
    combined_df = combined_df.join(df[asset], how='outer') 

  return combined_df

# Example usage
stables_final_df = create_combined_df(stables_dfs) 
asas_final_df = create_combined_df(asas_dfs)

In [14]:
stables_final_df.to_csv(f'{MONTH_DIR}/Results/stablecoins_24.csv')

In [30]:
asas_final_df.to_csv(f'{MONTH_DIR}/Results/asas_jan.csv')

In [13]:
algo_price = pd.read_csv(f'{MONTH_DIR}/algorand_historical_data.csv')
algo_price['snapped_at'] = pd.to_datetime(algo_price['snapped_at'])
price_feed = algo_price[(algo_price['snapped_at'] >= start_date) & (algo_price['snapped_at'] <= end_date)]
price_feed[['snapped_at', 'price']].to_csv(f'{MONTH_DIR}/Results/ALGO_dec.csv', index=False)

In [16]:
asa_volumes = pd.read_csv(f'{MONTH_DIR}/asa_volumes.csv')
asa_volumes['dt'] = pd.to_datetime(asa_volumes['dt'])
asa_volumes.columns.values[1:]
asas_final_df.reset_index(inplace=True)
asas_final_df['date'] = pd.to_datetime(asas_final_df['date'])

In [19]:
asa_volumes = pd.read_csv(f'{MONTH_DIR}/asa_volumes.csv')
asa_volumes['dt'] = pd.to_datetime(asa_volumes['dt'])


def multiply_columns(df1, df2):
  """
  Joins price and volume dataframes to convert into ALGO volumes

  Args:
    df1: Volumes Dataframe in native Currency
    df2: Asset/ALGO Price DataFrame.

  Returns:
    A merged DataFrame with ALGO Volumes
  """

  # Merge DataFrames on 'dt' and 'date'
  merged_df = df1.merge(df2, left_on='dt', right_on='date', suffixes=('_df1', '_df2'))

  # Calculate product of corresponding columns
  columns_to_multiply = df1.columns.values[1:]
  for col in columns_to_multiply:
      merged_df[col] = merged_df[col + '_df1'] * merged_df[col + '_df2']

  # Drop unnecessary columns
  merged_df.drop(columns=[col + '_df1' for col in columns_to_multiply] + 
                  [col + '_df2' for col in columns_to_multiply], 
             inplace=True)

  # Rename 'dt' column to 'date'
  merged_df.rename(columns={'dt': 'date'}, inplace=True)

  return merged_df


result_df = multiply_columns(asa_volumes, asas_final_df)

result_df

,date,index,date,ora,niko,talgo,vote,tiny,gobtc,silver,...,monko,opul,gold,chip,malgo,coop,avoi,goeth,daffir,calgo


In [45]:
def price_merge(algo_volumes, algo_price):
  """
  Joins volumes and algo price dataframes to convert into USD volume

  Args:
    df1: Volumes Dataframe in ALGO
    df2: ALGO/USD Price DataFrame.

  Returns:
    A merged DataFrame with ALGO Volumes
  """

    # Merge DataFrames on 'dt' and 'date'
  merged_df = algo_volumes.merge(algo_price[['snapped_at', 'price']], left_on='date', right_on='snapped_at')

  merged_df.drop('snapped_at', axis=1, inplace=True)
  # Calculate product of corresponding columns
  merged_df['usd_vol'] = merged_df['algo_vol']*merged_df['price']

  # Rename 'dt' column to 'date'
  #merged_df.rename(columns={'dt': 'date'}, inplace=True)

  return merged_df


stables = pd.read_csv(f'{MONTH_DIR}/stables.csv')
algorand = pd.read_csv(f'{MONTH_DIR}/algorand_historical_data.csv')

stables['dt'] = pd.to_datetime(stables['dt'])
algorand['snapped_at'] = pd.to_datetime(algorand['snapped_at']).dt.date

result_df = multiply_columns(stables, stables_final_df)
result_df['algo_vol'] = result_df[result_df.columns.values[1:]].sum(axis=1)

result_df['date'] = pd.to_datetime(result_df['date']).dt.date

usd = price_merge(result_df, algorand)


In [33]:
result_df.to_csv(f'{MONTH_DIR}/Results/asas_algo_volumes.csv', index=False)

In [24]:
# HAFN

def get_price_feed(ticker):
  """
  Fetches the price feed for the specified ticker for December 2024 
  using the yfinance library.

  Args:
    ticker: The ticker symbol (e.g., "AFNUSD=X").

  Returns:
    A pandas DataFrame containing the historical data for the specified period.
  """

  try:
    # Define start and end dates, consider forex does not operate 24/7
    start_date = "2025-01-01"
    end_date = "2025-01-31"

    # Download historical data using yfinance
    data = yf.download(ticker, start=start_date, end=end_date)
    return data
  except Exception as e:
    print(f"Error fetching data for {ticker}: {e}")
    return None

# Example usage
ticker = "AFNUSD=X"
december_data = get_price_feed(ticker)


[*********************100%%**********************]  1 of 1 completed


In [25]:
december_data.to_csv(f'{MONTH_DIR}/Results/HAFN_dec.csv')